In [1]:
import os, datetime
import pandas as pd
import numpy as np
from collections import defaultdict
from nltk.tokenize import sent_tokenize

# 1. Load matched items by LiAnS

In [2]:
df = pd.read_pickle("/work/qi.yu/notebook/rev_feat_ana_doc/doc/scripts_to_submit/data/matched_items_2023-04-21.pkl")
df.head()

,source,id,text,text_cleaned,causal,consecutive,adversative,concessive,conditional,common_ground,...,scalar_particle,economy,identity,legal,morality,policy,politics,public_opinion,security,welfare
0,0,BildOnline_02122014_1230Uhr73.xml,Sehen gut aus und tun Gutes. Die Bremer Kult-T...,Sehen gut aus und tun Gutes. Die Bremer Kult-T...,[],[],[],[],[wenn],[],...,[],[],[],[Drittstaat],[],"[Abschiebung, Drittstaat]","[Regierung, Regierung]",[],[kriminell],[]
1,0,BildOnline_19122014_2030Uhr39.xml,Jahrelang war eine Rückkehr der Wohnschiffe fü...,Jahrelang war eine Rückkehr der Wohnschiffe fü...,[denn],[],[],[],[wenn],[],...,[],[],[Nation],[],"[Solidarität, Solidarität]",[],[Kanzlerin],[],[],[]
2,0,BildOnline_19122014_0013Uhr46.xml,Man kann‘s ja mal versuchen... Die Chemnitzer ...,Man kann's ja mal versuchen... Die Chemnitzer ...,[],[],[],[],[],[],...,[],[],[],[],[],[Land],[],[],[],[]
3,0,BildOnline_24062014_1130Uhr94.xml,Jetzt wird es Ernst an der ehemaligen Gerhart-...,Jetzt wird es Ernst an der ehemaligen Gerhart-...,[da],[also],[jedoch],[zwar],"[wenn, wenn, wenn, wenn]",[],...,[],[],[national],[],[],"[Flüchtlingszahl, Land, Flüchtlingspolitik, Land]","[Kanzlerin, Partei, GroKo, Kanzlerin, Kanzleri...",[Stimmung],[Gewalt],[]
4,0,BildOnline_02072014_1745Uhr128.xml,Im Streit um die besetzte Flüchtlings-Schule i...,Im Streit um die besetzte Flüchtlings-Schule i...,[],[],[],[],[wenn],[],...,[],"[Kosten, Kosten, Kosten, Kosten]",[],[],[],"[Bund, Kommune]","[Bund, Kommune]",[],[],[]


# 2. Get most representative lexical items of each topic framing feature using pmi-freq

See Section 6.1 of the original paper for a detailed description of the implementation.

In [3]:
def get_matched_items(feature):
    feature = feature
    
    all_sources = []
    all_matches = []
    
    for idx, row in df.iterrows():
        if row[feature]:
            for match in row[feature]:
                all_matches.append(match)
                all_sources.append(row["source"])
                
    df_matched_items = pd.DataFrame({
        "source": all_sources,
        "match": all_matches
    })
    
    return df_matched_items

In [4]:
def pmi(dff, x, y):
    df = dff.copy()
    df['f_x'] = df.groupby(x)[x].transform('count')
    df['f_y'] = df.groupby(y)[y].transform('count')
    df['f_xy'] = df.groupby([x, y])[x].transform('count')
    df['pmi'] = np.log(len(df.index) * df['f_xy'] / (df['f_x'] * df['f_y']) )
    df['pmi_freq'] = np.log(df["f_y"]) * np.log(len(df.index) * df['f_xy'] / (df['f_x'] * df['f_y']) ) ###
    
    df.loc[df["f_y"] < 50 , "pmi_freq"] = 0 ###
    
    return df

## 2.1 Security

### 2.1.1 Compute PMI-freq

In [5]:
df_matched_items = get_matched_items("security")
print(df_matched_items.value_counts("source"))
df_matched_items

source
0    28521
1    15745
2    10500
dtype: int64


,source,match
0,0,kriminell
1,0,Gewalt
2,0,bedrohen
3,0,drohen
4,0,Verfolgung
...,...,...
54761,2,verfassungswidrig
54762,2,gefährlich
54763,2,Polizei
54764,2,Polizei


In [6]:
df_pmi = pmi(df_matched_items, "source", "match")
df_pmi.head()

,source,match,f_x,f_y,f_xy,pmi,pmi_freq
0,0,kriminell,28521,593,323,0.044887,0.286611
1,0,Gewalt,28521,1379,615,-0.155063,-1.120966
2,0,bedrohen,28521,477,229,-0.081366,-0.501824
3,0,drohen,28521,1764,837,-0.093086,-0.695851
4,0,Verfolgung,28521,574,182,-0.496194,-3.152135


### 2.1.2 Results
(as reported in Section 6.1 of the paper)

#### BILD:

In [7]:
df_pmi_bild = df_pmi[df_pmi["source"]==0].drop_duplicates().sort_values(by='pmi_freq', ascending=False)
df_pmi_bild.iloc[:10, ]

,source,match,f_x,f_y,f_xy,pmi,pmi_freq
72,0,ISIS,28521,522,517,0.642804,4.022455
30,0,U-Haft,28521,133,131,0.637277,3.116508
195,0,Körperverletzung,28521,507,383,0.371953,2.316713
621,0,Brandstiftung,28521,472,347,0.344775,2.122771
14,0,Totschlag,28521,153,119,0.401114,2.017781
31,0,Haft,28521,649,453,0.292888,1.896579
100,0,prügeln,28521,85,66,0.399432,1.774539
24,0,Volksverhetzung,28521,143,106,0.353023,1.752000
181,0,Gewahrsam,28521,112,83,0.352771,1.664548
924,0,tatverdächtig,28521,104,73,0.298497,1.386339


#### FAZ:

In [8]:
df_pmi_faz = df_pmi[df_pmi["source"]==1].drop_duplicates().sort_values(by='pmi_freq', ascending=False)
df_pmi_faz.iloc[:10, ]

,source,match,f_x,f_y,f_xy,pmi,pmi_freq
28697,1,Verfolgte,15745,223,135,0.744650,4.026449
28754,1,Menschenschmuggel,15745,105,59,0.670124,3.118730
28574,1,Verfolgung,15745,574,255,0.435181,2.764543
28567,1,unbegleitete_minderjaehrige,15745,508,224,0.427711,2.664847
28556,1,Krieg,15745,1654,666,0.336885,2.496635
28539,1,illegal,15745,2040,801,0.311703,2.375393
28566,1,minderjährig,15745,1270,504,0.322351,2.303768
28681,1,Bedrohung,15745,299,128,0.398133,2.269537
28555,1,Minderjährige,15745,525,214,0.349124,2.186706
28580,1,Menschenschmuggler,15745,134,58,0.409150,2.003951


### SZ:

In [9]:
df_pmi_sz = df_pmi[df_pmi["source"]==2].drop_duplicates().sort_values(by='pmi_freq', ascending=False)
df_pmi_sz.iloc[:10, ]

,source,match,f_x,f_y,f_xy,pmi,pmi_freq
44312,2,Schutzstatus,10500,268,91,0.571567,3.195623
44680,2,inhaftieren,10500,65,24,0.655361,2.735730
45661,2,Asylmissbrauch,10500,55,20,0.640093,2.565068
44423,2,Minderjährige,10500,525,151,0.405576,2.540283
44317,2,Küstenwache,10500,1101,289,0.314147,2.200276
44386,2,Menschenschmuggler,10500,134,40,0.442734,2.168440
44323,2,Schleuser,10500,1196,297,0.258689,1.833258
44330,2,Terrorismus,10500,264,70,0.324240,1.807948
44303,2,Gefahr,10500,1121,278,0.257339,1.807028
44422,2,Bürgerkrieg,10500,557,142,0.284956,1.801654


## 2.2 Morality

### 2.2.1 Compute PMI-freq

In [10]:
df_matched_items = get_matched_items("morality")
print(df_matched_items.value_counts("source"))
df_matched_items

source
1    14506
0    10444
2     7245
dtype: int64


,source,match
0,0,Solidarität
1,0,Solidarität
2,0,humanitär
3,0,Familiennachzug
4,0,Familiennachzug
...,...,...
32190,2,menschenwürdig
32191,2,Flüchtlingshilfe
32192,2,engagieren
32193,2,Flüchtlingshilfe


In [11]:
df_pmi = pmi(df_matched_items, "source", "match")
print(df_pmi.shape)
df_pmi.head()

(32195, 7)


,source,match,f_x,f_y,f_xy,pmi,pmi_freq
0,0,Solidarität,10444,1157,304,-0.210775,-1.486716
1,0,Solidarität,10444,1157,304,-0.210775,-1.486716
2,0,humanitär,10444,1399,393,-0.143920,-1.042485
3,0,Familiennachzug,10444,1091,394,0.107284,0.750439
4,0,Familiennachzug,10444,1091,394,0.107284,0.750439


### 2.2.2 Results
(as reported in Section 6.1 of the paper)

#### BILD:

In [12]:
df_pmi_bild = df_pmi[df_pmi["source"]==0].drop_duplicates().sort_values(by='pmi_freq', ascending=False)
df_pmi_bild.iloc[:10, ]

,source,match,f_x,f_y,f_xy,pmi,pmi_freq
127,0,fremdenfeindlich,10444,642,357,0.538931,3.483967
250,0,Anfeindung,10444,71,41,0.576676,2.458184
263,0,ausländerfeindlich,10444,102,56,0.526162,2.433487
20,0,Hartz,10444,275,135,0.414287,2.326956
51,0,Obergrenze,10444,1536,666,0.290136,2.128711
69,0,Menschlichkeit,10444,202,96,0.381864,2.027036
240,0,Mitmenschlichkeit,10444,52,28,0.506744,2.002270
7,0,Toleranz,10444,241,109,0.332334,1.822787
648,0,Fremdenfeindlichkeit,10444,182,81,0.316226,1.645642
200,0,Hilfsorganisation,10444,1050,428,0.228361,1.588606


#### FAZ:

In [13]:
df_pmi_faz = df_pmi[df_pmi["source"]==1].drop_duplicates().sort_values(by='pmi_freq', ascending=False)
df_pmi_faz.iloc[:10, ]

,source,match,f_x,f_y,f_xy,pmi,pmi_freq
10454,1,ehrenamtlich,14506,1159,687,0.274270,1.935062
10544,1,moralisch,14506,361,223,0.315543,1.858192
10785,1,Wirtschaftsmigrant,14506,112,74,0.382815,1.806312
10624,1,Völkerrecht,14506,85,57,0.397649,1.766615
10687,1,Verpflichtung,14506,402,234,0.256118,1.535798
10609,1,Empathie,14506,68,44,0.361931,1.527170
10625,1,Migrationspolitik,14506,270,158,0.261422,1.463550
10472,1,Willkommenskultur,14506,478,262,0.195983,1.209136
11219,1,Punktesystem,14506,56,34,0.298258,1.200592
10513,1,gemeinnützig,14506,150,85,0.229265,1.148762


#### SZ:

In [14]:
df_pmi_sz = df_pmi[df_pmi["source"]==2].drop_duplicates().sort_values(by='pmi_freq', ascending=False)
df_pmi_sz.iloc[:10, ]

,source,match,f_x,f_y,f_xy,pmi,pmi_freq
25362,2,human,7245,120,54,0.692992,3.317693
25098,2,Humanität,7245,174,65,0.506832,2.614772
25016,2,Existenzminimum,7245,81,29,0.464346,2.040546
25029,2,Konvention,7245,184,61,0.387438,2.020463
25083,2,Solidarität,7245,1157,342,0.272725,1.923686
24992,2,Quote,7245,806,240,0.280055,1.874150
24976,2,menschenwürdig,7245,294,89,0.296556,1.685501
25360,2,solidarisch,7245,260,79,0.300266,1.669683
24972,2,Not,7245,954,269,0.225547,1.547404
25052,2,Familiennachzug,7245,1091,303,0.210382,1.471593
